In [14]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_ta as ta

nifty_data = pd.read_csv("nifty_data.csv")

nifty_data['Date'] = pd.to_datetime(nifty_data['Date'], format="%Y-%m-%d") 
nifty_data.set_index('Date', inplace=True)

nifty_data['SMA_50'] = nifty_data['Close'].rolling(window=50).mean()
nifty_data['SMA_200'] = nifty_data['Close'].rolling(window=200).mean()

nifty_data['EMA_12'] = nifty_data['Close'].ewm(span=12, adjust=False).mean()
nifty_data['EMA_26'] = nifty_data['Close'].ewm(span=26, adjust=False).mean()

nifty_data.ta.macd(close = 'Close', fast=12, slow=26, signal=9, append=True)

nifty_data.ta.rsi(close='Close', length=14, append=True)

nifty_data = nifty_data.iloc[200:]
print(nifty_data)

                    Open          High           Low         Close  \
Date                                                                 
2014-04-07   6694.250000   6725.149902   6650.399902   6695.049805   
2014-04-09   6722.000000   6808.700195   6705.100098   6796.200195   
2014-04-10   6803.049805   6819.049805   6777.299805   6796.399902   
2014-04-11   6758.350098   6789.350098   6743.149902   6776.299805   
2014-04-15   6792.700195   6813.399902   6711.750000   6733.100098   
...                  ...           ...           ...           ...   
2024-07-12  24387.949219  24592.199219  24331.150391  24502.150391   
2024-07-15  24587.599609  24635.050781  24522.750000  24586.699219   
2024-07-16  24615.900391  24661.250000  24587.650391  24613.000000   
2024-07-18  24543.800781  24837.750000  24504.449219  24800.849609   
2024-07-19  24853.800781  24854.800781  24508.150391  24530.900391   

               Adj Close  Volume        SMA_50       SMA_200        EMA_12  \
Date       

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [4]:
X_nifty = nifty_data[['Close', 'SMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'EMA_12', 'EMA_26']]
y_nifty = nifty_data['Close']

X_train_nifty, X_test_nifty, y_train_nifty, y_test_nifty = train_test_split(X_nifty, y_nifty, test_size=0.2, shuffle=False)

print("X_train_nifty shape:", X_train_nifty.shape)
print("X_test_nifty_shape:", X_test_nifty.shape)
print("y_train_nifty_shape:", y_test_nifty.shape)
print("y_test_nifty_shape:", y_test_nifty.shape)

X_train_nifty shape: (2018, 9)
X_test_nifty_shape: (505, 9)
y_train_nifty_shape: (505,)
y_test_nifty_shape: (505,)


In [5]:
scaler = MinMaxScaler()
X_train_nifty_scaled = scaler.fit_transform(X_train_nifty)
X_test_nifty_scaled = scaler.transform(X_test_nifty)
X_train_nifty_dropped = X_train_nifty.dropna()
y_train_nifty_dropped = y_train_nifty[X_train_nifty_dropped.index]

In [6]:
lr_model_nifty = LinearRegression()
lr_model_nifty.fit(X_train_nifty_scaled, y_train_nifty)

LinearRegression()

In [7]:
lr_predictions_nifty = lr_model_nifty.predict(X_test_nifty_scaled)

mse_lr_nifty = mean_squared_error(y_test_nifty, lr_predictions_nifty)
mae_lr_nifty = mean_absolute_error(y_test_nifty, lr_predictions_nifty)
print(mse_lr_nifty)
print("\n", mae_lr_nifty)

2.8042241756253135e-24

 6.411487402597276e-13


In [17]:
tomorrow = pd.Timestamp.today() + pd.Timedelta(days=1)
next_week = pd.Timestamp.today() + pd.Timedelta(days=7)
next_month = pd.Timestamp.today() + pd.DateOffset(months=1)
next_year = pd.Timestamp.today() + pd.DateOffset(years = 1)

future_dates = pd.DataFrame({'Date': [tomorrow, next_week, next_month, next_year]})
future_dates.set_index('Date', inplace = True)
future_dates.index.freq = nifty_data.index.freq
print(nifty_data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA_50',
       'SMA_200', 'EMA_12', 'EMA_26', 'MACD_12_26_9', 'MACDh_12_26_9',
       'MACDs_12_26_9', 'RSI_14'],
      dtype='object')


In [22]:
latest_nifty_features = nifty_data.iloc[-1][['Close', 'SMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'EMA_12', 'EMA_26']]
future_nifty_features = pd.DataFrame(index = future_dates.index)
future_nifty_features[['Close', 'SMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'EMA_12', 'EMA_26']] = latest_nifty_features.values
print(future_nifty_features)

                                   Close        SMA_50       SMA_200  \
Date                                                                   
2024-08-30 21:20:48.958086  24530.900391  23356.534922  21719.351201   
2024-09-05 21:20:48.958086  24530.900391  23356.534922  21719.351201   
2024-09-29 21:20:48.958086  24530.900391  23356.534922  21719.351201   
2025-08-29 21:20:48.958086  24530.900391  23356.534922  21719.351201   

                               RSI_14  MACD_12_26_9  MACDh_12_26_9  \
Date                                                                 
2024-08-30 21:20:48.958086  65.420695    361.320392      -5.367287   
2024-09-05 21:20:48.958086  65.420695    361.320392      -5.367287   
2024-09-29 21:20:48.958086  65.420695    361.320392      -5.367287   
2025-08-29 21:20:48.958086  65.420695    361.320392      -5.367287   

                            MACDs_12_26_9        EMA_12        EMA_26  
Date                                                                   
20

In [24]:
future_nifty_features_scaled = scaler.transform(future_nifty_features)

In [26]:
future_nifty_predictions = lr_model_nifty.predict(future_nifty_features_scaled)

In [27]:
print("Nifty Predictions:")
for date, prediction in zip(future_dates.index, future_nifty_predictions):
    print(f"{date}: {prediction}")

Nifty Predictions:
2024-08-30 21:20:48.958086: 24530.900390625
2024-09-05 21:20:48.958086: 24530.900390625
2024-09-29 21:20:48.958086: 24530.900390625
2025-08-29 21:20:48.958086: 24530.900390625
